In [1]:
import org.apache.spark.sql.types.{StructType, StringType, TimestampType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame


In [2]:
println(s"Current spark version is ${spark.version}")

Current spark version is 2.4.4


In [3]:
val inputStreamPath = "/home/jovyan/data/events-stream"
val modelPath = "/home/jovyan/models/spark-random_forest-ml-model"

val dataSchema = new StructType()
    .add("tweet", StringType)
    .add("timestamp", TimestampType)

val inputDF = spark
    .readStream
    .schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .json(inputStreamPath)

inputStreamPath = /home/jovyan/data/events-stream
modelPath = /home/jovyan/models/spark-random_forest-ml-model
dataSchema = StructType(StructField(tweet,StringType,true), StructField(timestamp,TimestampType,true))
inputDF = [tweet: string, timestamp: timestamp]


[tweet: string, timestamp: timestamp]

In [4]:
// загружаем модель в объект
val model = PipelineModel.load(modelPath)

model = pipeline_d732f58e5dab


pipeline_d732f58e5dab

In [5]:
// функция для получения элемента 1 из probability
val getProbability = udf((p: org.apache.spark.ml.linalg.Vector) => p.toArray(1))

getProbability = UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))


UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))

In [7]:
// запускаем модель для каждой микро-пачки
// выводим элемент 1 из "probability"
inputDF.writeStream.foreachBatch { (batchDF: DataFrame, batchId: Long) =>
    println (batchId)
    // запускаем модель
    model.transform(batchDF)
        .select(getProbability($"probability").alias("probability_1"))
        .show(false)
}
.start()

org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@4ece0124

0
+-------------------+
|probability_1      |
+-------------------+
|0.5028275172582966 |
|0.5028275172582966 |
|0.5028275172582966 |
|0.5028275172582966 |
|0.5028275172582966 |
|0.40431175924111457|
|0.5028275172582966 |
|0.5028275172582966 |
|0.5028275172582966 |
|0.549917097257528  |
|0.46705222887601144|
|0.5028275172582966 |
|0.5028275172582966 |
|0.5028275172582966 |
|0.40431175924111457|
|0.5028275172582966 |
|0.5028275172582966 |
|0.5714848214956812 |
|0.5028275172582966 |
|0.5028275172582966 |
+-------------------+

1
+------------------+
|probability_1     |
+------------------+
|0.5028275172582966|
|0.5028275172582966|
|0.5028275172582966|
|0.641610713065007 |
|0.3746397962453512|
|0.5028275172582966|
|0.5028275172582966|
|0.5074316333414951|
|0.5028275172582966|
|0.5028275172582966|
|0.5028275172582966|
|0.5028275172582966|
|0.5028275172582966|
|0.5028275172582966|
|0.5028275172582966|
|0.5028275172582966|
|0.5028275172582966|
|0.5349292471185025|
|0.5028275172582966|
|0.50

18
+-------------------+
|probability_1      |
+-------------------+
|0.5028275172582966 |
|0.40431175924111457|
|0.5028275172582966 |
|0.5028275172582966 |
|0.5028275172582966 |
|0.549917097257528  |
|0.5028275172582966 |
|0.5028275172582966 |
|0.549917097257528  |
|0.5028275172582966 |
|0.5028275172582966 |
+-------------------+

19
+-------------------+
|probability_1      |
+-------------------+
|0.48662350440212565|
|0.5028275172582966 |
|0.5028275172582966 |
|0.5028275172582966 |
|0.5028275172582966 |
|0.45679723314777404|
|0.5028275172582966 |
|0.5028275172582966 |
|0.5028275172582966 |
|0.5028275172582966 |
|0.5028275172582966 |
|0.5028275172582966 |
|0.5714848214956812 |
|0.45679723314777404|
|0.5028275172582966 |
|0.5028275172582966 |
|0.5028275172582966 |
|0.5349292471185025 |
|0.641610713065007  |
|0.5028275172582966 |
+-------------------+
20
+------------------+
|probability_1     |
+------------------+
|0.5028275172582966|
|0.5028275172582966|
|0.5028275172582966|
|0.502

+-------------------+
|probability_1      |
+-------------------+
|0.5028275172582966 |
|0.43877432910411035|
|0.5028275172582966 |
|0.5028275172582966 |
|0.4529542882105761 |
|0.5028275172582966 |
|0.5028275172582966 |
|0.5028275172582966 |
|0.5028275172582966 |
|0.5028275172582966 |
|0.43953392440289424|
|0.5028275172582966 |
|0.5028275172582966 |
|0.40431175924111457|
+-------------------+

38
+-------------------+
|probability_1      |
+-------------------+
|0.5028275172582966 |
|0.5028275172582966 |
|0.5028275172582966 |
|0.641610713065007  |
|0.4529542882105761 |
|0.5028275172582966 |
|0.5028275172582966 |
|0.5081912286402789 |
|0.5028275172582966 |
|0.5028275172582966 |
|0.5028275172582966 |
|0.5028275172582966 |
|0.45140133924034603|
|0.46705222887601144|
|0.42451302529307167|
+-------------------+

39
+-------------------+
|probability_1      |
+-------------------+
|0.5028275172582966 |
|0.5028275172582966 |
|0.40431175924111457|
|0.5028275172582966 |
|0.45679723314777404|
|0

In [10]:
// вывод кол-ва негативных и позитивных твитов за 10 сек скользящим окном
inputDF.writeStream.foreachBatch { (batchDF: DataFrame, batchId: Long) =>
    println (batchId)
    model.transform(batchDF)
        .withColumn("Negative", when($"prediction" === 1, 1).otherwise(1))
        .withColumn("Positive", when($"prediction" === 0, 1).otherwise(0))
        .withWatermark("timestamp", "15 seconds")
        .groupBy(window($"timestamp", "10 seconds", "2 seconds"))
        .agg(sum($"Negative").alias("Negative"), sum($"Positive").alias("Positive"))
        .orderBy($"window")
        .show(false)
}
.start()

0


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@3c147ee3

+------------------------------------------+--------+--------+
|window                                    |Negative|Positive|
+------------------------------------------+--------+--------+
|[2020-02-08 00:40:40, 2020-02-08 00:40:50]|20      |3       |
|[2020-02-08 00:40:42, 2020-02-08 00:40:52]|20      |3       |
|[2020-02-08 00:40:44, 2020-02-08 00:40:54]|20      |3       |
|[2020-02-08 00:40:46, 2020-02-08 00:40:56]|20      |3       |
|[2020-02-08 00:40:48, 2020-02-08 00:40:58]|20      |3       |
+------------------------------------------+--------+--------+

1
+------------------------------------------+--------+--------+
|window                                    |Negative|Positive|
+------------------------------------------+--------+--------+
|[2020-02-08 00:40:46, 2020-02-08 00:40:56]|23      |1       |
|[2020-02-08 00:40:48, 2020-02-08 00:40:58]|23      |1       |
|[2020-02-08 00:40:50, 2020-02-08 00:41:00]|23      |1       |
|[2020-02-08 00:40:52, 2020-02-08 00:41:02]|23      

15
+------------------------------------------+--------+--------+
|window                                    |Negative|Positive|
+------------------------------------------+--------+--------+
|[2020-02-08 00:42:04, 2020-02-08 00:42:14]|17      |4       |
|[2020-02-08 00:42:06, 2020-02-08 00:42:16]|17      |4       |
|[2020-02-08 00:42:08, 2020-02-08 00:42:18]|17      |4       |
|[2020-02-08 00:42:10, 2020-02-08 00:42:20]|17      |4       |
|[2020-02-08 00:42:12, 2020-02-08 00:42:22]|17      |4       |
+------------------------------------------+--------+--------+

16
+------------------------------------------+--------+--------+
|window                                    |Negative|Positive|
+------------------------------------------+--------+--------+
|[2020-02-08 00:42:10, 2020-02-08 00:42:20]|20      |5       |
|[2020-02-08 00:42:12, 2020-02-08 00:42:22]|20      |5       |
|[2020-02-08 00:42:14, 2020-02-08 00:42:24]|20      |5       |
|[2020-02-08 00:42:16, 2020-02-08 00:42:26]|20  

30
+------------------------------------------+--------+--------+
|window                                    |Negative|Positive|
+------------------------------------------+--------+--------+
|[2020-02-08 00:43:20, 2020-02-08 00:43:30]|16      |2       |
|[2020-02-08 00:43:22, 2020-02-08 00:43:32]|16      |2       |
|[2020-02-08 00:43:24, 2020-02-08 00:43:34]|16      |2       |
|[2020-02-08 00:43:26, 2020-02-08 00:43:36]|16      |2       |
|[2020-02-08 00:43:28, 2020-02-08 00:43:38]|16      |2       |
+------------------------------------------+--------+--------+

31
+------------------------------------------+--------+--------+
|window                                    |Negative|Positive|
+------------------------------------------+--------+--------+
|[2020-02-08 00:43:26, 2020-02-08 00:43:36]|19      |2       |
|[2020-02-08 00:43:28, 2020-02-08 00:43:38]|19      |2       |
|[2020-02-08 00:43:30, 2020-02-08 00:43:40]|19      |2       |
|[2020-02-08 00:43:32, 2020-02-08 00:43:42]|19  

45
+------------------------------------------+--------+--------+
|window                                    |Negative|Positive|
+------------------------------------------+--------+--------+
|[2020-02-08 00:44:38, 2020-02-08 00:44:48]|17      |3       |
|[2020-02-08 00:44:40, 2020-02-08 00:44:50]|17      |3       |
|[2020-02-08 00:44:42, 2020-02-08 00:44:52]|17      |3       |
|[2020-02-08 00:44:44, 2020-02-08 00:44:54]|17      |3       |
|[2020-02-08 00:44:46, 2020-02-08 00:44:56]|17      |3       |
+------------------------------------------+--------+--------+

46
+------------------------------------------+--------+--------+
|window                                    |Negative|Positive|
+------------------------------------------+--------+--------+
|[2020-02-08 00:44:42, 2020-02-08 00:44:52]|20      |3       |
|[2020-02-08 00:44:44, 2020-02-08 00:44:54]|20      |3       |
|[2020-02-08 00:44:46, 2020-02-08 00:44:56]|20      |3       |
|[2020-02-08 00:44:48, 2020-02-08 00:44:58]|20  

In [11]:
// эксперименты со скользящим окном
inputDF
    .withWatermark("timestamp", "15 seconds")
    .groupBy(window($"timestamp", "10 seconds", "2 seconds")).count()
    .writeStream.format("console").option("truncate", false)
    .start()

org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@42d739f6

-------------------------------------------
Batch: 0
-------------------------------------------
+------+-----+
|window|count|
+------+-----+
+------+-----+

-------------------------------------------
Batch: 1
-------------------------------------------
+------+-----+
|window|count|
+------+-----+
+------+-----+

-------------------------------------------
Batch: 2
-------------------------------------------
+------+-----+
|window|count|
+------+-----+
+------+-----+

-------------------------------------------
Batch: 3
-------------------------------------------
+------+-----+
|window|count|
+------+-----+
+------+-----+

-------------------------------------------
Batch: 4
-------------------------------------------
+------------------------------------------+-----+
|window                                    |count|
+------------------------------------------+-----+
|[2020-02-08 00:40:42, 2020-02-08 00:40:52]|20   |
|[2020-02-08 00:40:40, 2020-02-08 00:40:50]|20   |
+---------------

-------------------------------------------
Batch: 22
-------------------------------------------
+------------------------------------------+-----+
|window                                    |count|
+------------------------------------------+-----+
|[2020-02-08 00:42:14, 2020-02-08 00:42:24]|39   |
|[2020-02-08 00:42:16, 2020-02-08 00:42:26]|39   |
|[2020-02-08 00:42:18, 2020-02-08 00:42:28]|39   |
+------------------------------------------+-----+

-------------------------------------------
Batch: 23
-------------------------------------------
+------------------------------------------+-----+
|window                                    |count|
+------------------------------------------+-----+
|[2020-02-08 00:42:22, 2020-02-08 00:42:32]|30   |
|[2020-02-08 00:42:20, 2020-02-08 00:42:30]|30   |
+------------------------------------------+-----+

-------------------------------------------
Batch: 24
-------------------------------------------
+----------------------------------------